# Libraries&Function 

In [1]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features import degrees,  pagerank

from reskit.core import Transformer, Pipeliner

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def orig(x):
    return x

import copy

In [3]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict

import matrix_eig as me

# Train mosters
## Logistic Regression
### Chose the best model for each k

In [3]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism))]

weighters = [('origW', Transformer(orig))]

normalizers = [('origN', Transformer(orig))]

featurizers = [('origF', Transformer(me.orig_vec, collect=['X_vec'])),
               ('low0', Transformer(me.matrix_eig_0, collect=['X_vec'])),
               ('low5', Transformer(me.matrix_eig_5, collect=['X_vec'])),
               ('low10', Transformer(me.matrix_eig_10, collect=['X_vec'])),
               ('low20', Transformer(me.matrix_eig_20, collect=['X_vec'])),
               ('low30', Transformer(me.matrix_eig_30, collect=['X_vec'])),
               ('low40', Transformer(me.matrix_eig_40, collect=['X_vec'])),
               ('low50', Transformer(me.matrix_eig_50, collect=['X_vec'])),
               ('low60', Transformer(me.matrix_eig_60, collect=['X_vec'])),
               ('low70', Transformer(me.matrix_eig_70, collect=['X_vec'])),
               ('low80', Transformer(me.matrix_eig_80, collect=['X_vec'])),
               ('low90', Transformer(me.matrix_eig_90, collect=['X_vec'])),
               ('low100', Transformer(me.matrix_eig_100, collect=['X_vec'])),
               ('low110', Transformer(me.matrix_eig_110, collect=['X_vec'])),
               ('low120', Transformer(me.matrix_eig_120, collect=['X_vec'])),
               ('low130', Transformer(me.matrix_eig_130, collect=['X_vec'])),
               ('low140', Transformer(me.matrix_eig_140, collect=['X_vec'])),
               ('low150', Transformer(me.matrix_eig_150, collect=['X_vec'])),
               ('low160', Transformer(me.matrix_eig_160, collect=['X_vec'])),
               ('low170', Transformer(me.matrix_eig_170, collect=['X_vec'])),
               ('low180', Transformer(me.matrix_eig_180, collect=['X_vec'])),
               ('low190', Transformer(me.matrix_eig_190, collect=['X_vec'])),
               ('low200', Transformer(me.matrix_eig_200, collect=['X_vec'])),
               ('low210', Transformer(me.matrix_eig_210, collect=['X_vec'])),
               ('low220', Transformer(me.matrix_eig_220, collect=['X_vec'])),
               ('low230', Transformer(me.matrix_eig_230, collect=['X_vec'])),
               ('low240', Transformer(me.matrix_eig_240, collect=['X_vec'])),
               ('low250', Transformer(me.matrix_eig_250, collect=['X_vec'])),
               ('low260', Transformer(me.matrix_eig_260, collect=['X_vec'])),
               ('low263', Transformer(me.matrix_eig_263, collect=['X_vec']))
              ]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('LR', LogisticRegression())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
     LR=dict(
        C=[0.01, 0.05, 0.1] + [0.05*i for i in range(3, 21)],
        max_iter=[50, 100, 500],
        penalty=['l1', 'l2']
    )
    )

banned_combos = []

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,origN,origF,var_threshold,minmax,LR
1,UCLAsource,origW,origN,low0,var_threshold,minmax,LR
2,UCLAsource,origW,origN,low5,var_threshold,minmax,LR
3,UCLAsource,origW,origN,low10,var_threshold,minmax,LR
4,UCLAsource,origW,origN,low20,var_threshold,minmax,LR
5,UCLAsource,origW,origN,low30,var_threshold,minmax,LR
6,UCLAsource,origW,origN,low40,var_threshold,minmax,LR
7,UCLAsource,origW,origN,low50,var_threshold,minmax,LR
8,UCLAsource,origW,origN,low60,var_threshold,minmax,LR
9,UCLAsource,origW,origN,low70,var_threshold,minmax,LR


In [4]:
result = pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                          scoring=['roc_auc'],  results_file = "Results/Ver3.0/result_1.csv")

No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30


In [4]:
result = pd.read_csv("Results/Ver3.0/result_1.csv")

In [9]:
result

,Unnamed: 0,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,0,UCLAsource,origW,origN,origF,var_threshold,minmax,LR,0.607801,0.157715,"{'penalty': 'l2', 'C': 0.1, 'max_iter': 50}",0.593333,0.159332,[ 0.63333333 0.56 0.64 0.85 ...
1,1,UCLAsource,origW,origN,low0,var_threshold,minmax,LR,0.564894,0.139228,"{'penalty': 'l1', 'C': 0.75, 'max_iter': 500}",0.482000,0.143164,[ 0.6 0.56 0.36 0.6 0.6 0.6 0.4 0....
2,2,UCLAsource,origW,origN,low5,var_threshold,minmax,LR,0.596099,0.129598,"{'penalty': 'l1', 'C': 0.9, 'max_iter': 50}",0.572000,0.126554,[ 0.6 0.64 0.68 0.75 0.55 0.6 0.55 0....
3,3,UCLAsource,origW,origN,low10,var_threshold,minmax,LR,0.597695,0.168164,"{'penalty': 'l1', 'C': 0.8, 'max_iter': 50}",0.522333,0.154884,[ 0.63333333 0.6 0.64 0.7 ...
4,4,UCLAsource,origW,origN,low20,var_threshold,minmax,LR,0.624468,0.152592,"{'penalty': 'l1', 'C': 0.95, 'max_iter': 100}",0.539000,0.139675,[ 0.5 0.52 0.32 0.8 0.45 0.75 0.5 0....
5,5,UCLAsource,origW,origN,low30,var_threshold,minmax,LR,0.605851,0.201280,"{'penalty': 'l1', 'C': 0.9, 'max_iter': 100}",0.520000,0.120830,[ 0.5 0.6 0.6 0.65 0.5 0.65 0.3 0....
6,6,UCLAsource,origW,origN,low40,var_threshold,minmax,LR,0.638121,0.197559,"{'penalty': 'l1', 'C': 0.95, 'max_iter': 100}",0.576000,0.176136,[ 0.7 0.72 0.64 0.75 0.5 0.75 0.35 0....
7,7,UCLAsource,origW,origN,low50,var_threshold,minmax,LR,0.610461,0.148148,"{'penalty': 'l1', 'C': 0.75, 'max_iter': 50}",0.551000,0.132095,[ 0.6 0.64 0.72 0.65 0.65 0.55 0.4 0....
8,8,UCLAsource,origW,origN,low60,var_threshold,minmax,LR,0.625177,0.153770,"{'penalty': 'l1', 'C': 0.9, 'max_iter': 50}",0.557333,0.141388,[ 0.63333333 0.68 0.76 0.7 ...
9,9,UCLAsource,origW,origN,low70,var_threshold,minmax,LR,0.635461,0.148659,"{'penalty': 'l1', 'C': 0.85, 'max_iter': 50}",0.556000,0.157620,[ 0.6 0.48 0.88 0.7 0.6 0.5 0.45 0....


In [5]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism))]

weighters = [('origW', Transformer(orig))]

normalizers = [('origN', Transformer(orig))]

featurizers = [('origF', Transformer(me.orig_vec, collect=['X_vec'])),
               ('low0', Transformer(me.matrix_eig_0, collect=['X_vec'])),
               ('low5', Transformer(me.matrix_eig_5, collect=['X_vec'])),
               ('low10', Transformer(me.matrix_eig_10, collect=['X_vec'])),
               ('low20', Transformer(me.matrix_eig_20, collect=['X_vec'])),
               ('low30', Transformer(me.matrix_eig_30, collect=['X_vec'])),
               ('low40', Transformer(me.matrix_eig_40, collect=['X_vec'])),
               ('low50', Transformer(me.matrix_eig_50, collect=['X_vec'])),
               ('low60', Transformer(me.matrix_eig_60, collect=['X_vec'])),
               ('low70', Transformer(me.matrix_eig_70, collect=['X_vec'])),
               ('low80', Transformer(me.matrix_eig_80, collect=['X_vec'])),
               ('low90', Transformer(me.matrix_eig_90, collect=['X_vec'])),
               ('low100', Transformer(me.matrix_eig_100, collect=['X_vec'])),
               ('low110', Transformer(me.matrix_eig_110, collect=['X_vec'])),
               ('low120', Transformer(me.matrix_eig_120, collect=['X_vec'])),
               ('low130', Transformer(me.matrix_eig_130, collect=['X_vec'])),
               ('low140', Transformer(me.matrix_eig_140, collect=['X_vec'])),
               ('low150', Transformer(me.matrix_eig_150, collect=['X_vec'])),
               ('low160', Transformer(me.matrix_eig_160, collect=['X_vec'])),
               ('low170', Transformer(me.matrix_eig_170, collect=['X_vec'])),
               ('low180', Transformer(me.matrix_eig_180, collect=['X_vec'])),
               ('low190', Transformer(me.matrix_eig_190, collect=['X_vec'])),
               ('low200', Transformer(me.matrix_eig_200, collect=['X_vec'])),
               ('low210', Transformer(me.matrix_eig_210, collect=['X_vec'])),
               ('low220', Transformer(me.matrix_eig_220, collect=['X_vec'])),
               ('low230', Transformer(me.matrix_eig_230, collect=['X_vec'])),
               ('low240', Transformer(me.matrix_eig_240, collect=['X_vec'])),
               ('low250', Transformer(me.matrix_eig_250, collect=['X_vec'])),
               ('low260', Transformer(me.matrix_eig_260, collect=['X_vec'])),
               ('low263', Transformer(me.matrix_eig_263, collect=['X_vec']))
              ]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('LR_orig', LogisticRegression()),
               ('LR_0', LogisticRegression()),
               ('LR_5', LogisticRegression()),
               ('LR_10', LogisticRegression()),
               ('LR_20', LogisticRegression()),
               ('LR_30', LogisticRegression()),
               ('LR_40', LogisticRegression()),
               ('LR_50', LogisticRegression()),
               ('LR_60', LogisticRegression()),
               ('LR_70', LogisticRegression()),
               ('LR_80', LogisticRegression()),
               ('LR_90', LogisticRegression()),
               ('LR_100', LogisticRegression()),
               ('LR_110', LogisticRegression()),
               ('LR_120', LogisticRegression()),
               ('LR_130', LogisticRegression()),
               ('LR_140', LogisticRegression()),
               ('LR_150', LogisticRegression()),
               ('LR_160', LogisticRegression()),
               ('LR_170', LogisticRegression()),
               ('LR_180', LogisticRegression()),
               ('LR_190', LogisticRegression()),
               ('LR_200', LogisticRegression()),
               ('LR_210', LogisticRegression()),
               ('LR_220', LogisticRegression()),
               ('LR_230', LogisticRegression()),
               ('LR_240', LogisticRegression()),
               ('LR_250', LogisticRegression()),
               ('LR_260', LogisticRegression()),
               ('LR_263', LogisticRegression()),
              ]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
    LR_orig = dict(penalty= ['l2'], C= [0.1], max_iter= [50]),
    LR_0    = dict(penalty= ['l1'], C= [0.75], max_iter= [500]),
    LR_5    = dict(penalty= ['l1'], C= [0.9], max_iter= [50]),
    LR_10   = dict(penalty= ['l1'], C= [0.8], max_iter= [50]),
    LR_20   = dict(penalty= ['l1'], C= [0.95], max_iter= [100]),
    LR_30   = dict(penalty= ['l1'], C= [0.9], max_iter= [100]),
    LR_40   = dict(penalty= ['l1'], C= [0.95], max_iter= [100]),
    LR_50   = dict(penalty= ['l1'], C= [0.75], max_iter= [50]),
    LR_60   = dict(penalty= ['l1'], C= [0.9], max_iter= [50]),
    LR_70   = dict(penalty= ['l1'], C= [0.85], max_iter= [50]),
    LR_80   = dict(penalty= ['l2'], C= [0.3], max_iter= [50]),
    LR_90   = dict(penalty= ['l1'], C= [0.65], max_iter= [500]),
    LR_100  = dict(penalty= ['l1'], C= [0.65], max_iter= [100]),
    LR_110  = dict(penalty= ['l2'], C= [0.65], max_iter= [50]),
    LR_120  = dict(penalty= ['l2'], C= [0.01], max_iter= [50]),
    LR_130  = dict(penalty= ['l2'], C= [0.01], max_iter= [50]),
    LR_140  = dict(penalty= ['l2'], C= [0.01], max_iter= [50]), #New
    LR_150  = dict(penalty= ['l2'], C= [0.01], max_iter= [50]),
    LR_160  = dict(penalty= ['l2'], C= [0.05], max_iter= [50]),
    LR_170  = dict(penalty= ['l2'], C= [0.05], max_iter= [50]),
    LR_180  = dict(penalty= ['l1'], C= [0.35], max_iter= [100]),
    LR_190  = dict(penalty= ['l1'], C= [0.35], max_iter= [500]),
    LR_200  = dict(penalty= ['l2'], C= [0.15], max_iter= [50]),
    LR_210  = dict(penalty= ['l2'], C= [0.05], max_iter= [50]),
    LR_220  = dict(penalty= ['l2'], C= [0.01], max_iter= [50]),
    LR_230  = dict(penalty= ['l2'], C= [0.01], max_iter= [50]), #New
    LR_240  = dict(penalty= ['l2'], C= [0.01], max_iter= [50]),
    LR_250  = dict(penalty= ['l2'], C= [0.01], max_iter= [50]),
    LR_260  = dict(penalty= ['l2'], C= [0.15], max_iter= [50]), 
    LR_263  = dict(penalty= ['l2'], C= [0.1], max_iter= [50])  #New
    )

banned_combos = []

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

In [6]:
k = [0, 5]+ [10*i for i in range(1, 27)] + [263]
col = result.as_matrix(["Featurizers"])
col_f = np.delete(np.hstack(col), 0)

for n,fet in enumerate(col_f):
    k_tmp = copy.copy(k)
    k_tmp.remove(k[n])
    for i in k_tmp: banned_combos.append((fet, 'LR_' + str(i)))
        
for i in k: banned_combos.append(('origF', 'LR_' + str(i)))
    
for fet in col_f: banned_combos.append((fet, 'LR_orig'))

In [7]:
pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,origN,origF,var_threshold,minmax,LR_orig
1,UCLAsource,origW,origN,low0,var_threshold,minmax,LR_0
2,UCLAsource,origW,origN,low5,var_threshold,minmax,LR_5
3,UCLAsource,origW,origN,low10,var_threshold,minmax,LR_10
4,UCLAsource,origW,origN,low20,var_threshold,minmax,LR_20
5,UCLAsource,origW,origN,low30,var_threshold,minmax,LR_30
6,UCLAsource,origW,origN,low40,var_threshold,minmax,LR_40
7,UCLAsource,origW,origN,low50,var_threshold,minmax,LR_50
8,UCLAsource,origW,origN,low60,var_threshold,minmax,LR_60
9,UCLAsource,origW,origN,low70,var_threshold,minmax,LR_70


In [ ]:
num_epoch = 50

for epoch in range(1, num_epoch):

    grid_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=0)

    eval_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=epoch)

    
    
    pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
    #pipe.plan_table
    
    pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                 scoring=['roc_auc'], results_file = "Results/Ver3.7/result_" + str(epoch) + ".csv")

Removed previous results file -- Results/Ver3.7/result_1.csv.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30
Removed previous results file -- Results/Ver3.7/result_2.csv.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30
No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30


In [9]:
num_epoch = 100

for epoch in range(50, num_epoch):

    grid_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=0)

    eval_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=epoch)

    
    
    pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
    #pipe.plan_table
    
    pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                 scoring=['roc_auc'], results_file = "Results/Ver3.7/result_" + str(epoch) + ".csv")

No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30
No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30
No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/3